# **Clusterização (Agrupamento)**

Objetivo: médodo de aprendizagem não-supervisionada onde se busca encontrar padrões em dados brutos, de forma a obter grupos (*clusters*) com características distintas.

> **Funcionamento:**

<img src="https://github.com/OviedoVR/ML_especializacao/blob/main/assets/kmeans-working.png?raw=true" width="450px"/>

<br>

> **Tipos de clusterização:**

<img src="https://github.com/OviedoVR/ML_especializacao/blob/main/assets/clustering-types.png?raw=true" width="90%"/>

[10.1016/j.psychres.2023.115265](https://doi.org/10.1016/j.psychres.2023.115265)

Artigo: [data-clustering-python](https://builtin.com/data-science/data-clustering-python)


In [63]:
%%writefile custom_layout.py
import plotly.express as px

# Define a custom layout dictionary
custom_layout = {
    'width': 700,
    'height': 400,
    'paper_bgcolor': 'white',
    'plot_bgcolor': 'white',
    'font': {'family': 'Palatino', 'size': 12},
    'xaxis_showgrid': False,
    'yaxis_showgrid': False,
    'xaxis_linecolor': 'lightgray',
    'yaxis_linecolor': 'lightgray',
    'xaxis_zeroline': False,
    'xaxis': {'showline': True}
}

# Set the default template:
px.defaults.template = 'simple_white'

# Defining a color palette:
color_discrete_sequence = ['#bdbf09', '#2292a4', 'lightgray', '#CE4760', '#d96c06']

# Create a custom marker style dictionary
custom_marker_style = {
    'size': 10,            # Adjust the size as needed
    'line_width': 1,
    'opacity': 0.7         # Adjust the opacity as needed
}

Overwriting custom_layout.py


In [64]:
# -- Manipulação de dados:
import numpy as np
import pandas as pd

# -- Desligar warnings:
import warnings

# -- Visualização de dados:
import plotly.express as px                              # core library
import plotly.graph_objs as go                           # core library
from plotly.subplots import make_subplots                # core library
import matplotlib.pyplot as plt                          # auxiliar
import seaborn as sns                                    # auxiliar
from custom_layout import *

# -- Modelagem:
from sklearn.pipeline import Pipeline                    # pipelines
from sklearn.preprocessing import StandardScaler         # normalização
from sklearn.preprocessing import LabelEncoder           # codificação dados categóricos
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans                       # agrupamento
from sklearn.metrics import silhouette_score             # validação

## Pipeline

O pipeline segue os passos a seguir:

1. Criar o dataset da *features*
2. Criar uma amostra do dataset, em caso de datasets grandes (OPCIONAL)
3. Modelagem: agrupamento com K-Means
4. Realizar novas predições (OPCIONAL)
5. Exploração interativa dos clusters

Para cada etapa, temos funções a serem aplicadas.

**Funções**

In [65]:
def copy_dataframe(dataframe):
    #Creates a copy of the input dataframe (backup)

    copied_df = dataframe.copy()
    return copied_df
# --------------------------------------------------------------------- Normalização:
def scale_features(dataframe, selected_columns):
    """
    Normalizes the selected numercial columns of  a dataframe
    """
    features = dataframe[selected_columns]
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    scaled_df = pd.DataFrame(scaled_features, columns=selected_columns)

    return scaled_df
# ----------------------------------------------------------------------- Codificação:
def label_encode_features(dataframe, selected_columns):
    """
    Encodes the selected categorical columns of a dataframe
    """
    encoded_df = dataframe.copy()

    # Initialize the LabelEncoder
    label_encoder = LabelEncoder()

    # Apply the LabelEncoder to the selected columns
    for column in selected_columns:
        if column in encoded_df.columns:
            encoded_df[column] = label_encoder.fit_transform(encoded_df[column])

    return encoded_df
# --------------------------------------------------------------------------------- Escolha do k:
def elbow_silhouette_subplots(dataframe, max_clusters, template='simple_white'):
    """
    Generate elbow and silhouette score subplots for different numbers of clusters.
    Args:
      - dataframe (pd.DataFrame): The DataFrame containing the features.
      - max_clusters (int): The maximum number of clusters to consider.
      - template (str): Plotly template for the plots (default is 'simple_white').
    Returns:
      - None (displays the subplots).
    """
    # Suppress FutureWarnings
    warnings.simplefilter(action='ignore', category=FutureWarning)

    # Create empty lists to store inertia (within-cluster sum of squares) and silhouette scores
    inertias = []
    silhouette_scores = []

    # Iterate through different numbers of clusters and fit K-Means
    for n_clusters in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=n_clusters, random_state=35)
        cluster_labels = kmeans.fit_predict(dataframe)

        # Calculate the inertia and silhouette score
        inertias.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(dataframe, cluster_labels))

    # Convert the range to a list
    num_clusters = list(range(2, max_clusters + 1))

    # Create subplots with two columns
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Elbow Method', 'Silhouette Score'])

    # Add the elbow plot to the first subplot
    fig.add_trace(go.Scatter(x=num_clusters, y=inertias, mode='lines+markers', name='Inertia'), row=1, col=1)
    fig.update_xaxes(title_text='Number of Clusters', row=1, col=1)
    fig.update_yaxes(title_text='Inertia', row=1, col=1)

    # Add the silhouette score plot to the second subplot
    fig.add_trace(go.Scatter(x=num_clusters, y=silhouette_scores, mode='lines+markers', name='Silhouette Score'), row=1, col=2)
    fig.update_xaxes(title_text='Number of Clusters', row=1, col=2)
    fig.update_yaxes(title_text='Silhouette Score', row=1, col=2)

    # Update the layout
    fig.update_layout(title='Elbow Method and Silhouette Score', template=template)
    fig.show()
# --------------------------------------------------------------------------------- K-means:
def apply_kmeans(dataframe, k):
    """
    Apply K-Means clustering to the input dataframe.
    Args:
      - dataframe (pd.DataFrame): The DataFrame containing the features.
      - k (int): The number of clusters to create.
    Returns:
      - labels (pd.Series): A Series containing cluster labels for each row.
    """
    kmeans = KMeans(n_clusters=k, random_state=35)
    labels = kmeans.fit_predict(dataframe)

    return labels

## **Modelagem**

### ***Dados***

In [66]:
!wget -q 'https://raw.githubusercontent.com/OviedoVR/ML_especializacao/main/dados/campaings.csv' -O 'campaings.csv'

**Dicionário de dados**

| Field             | Description                                               |
|-------------------|-----------------------------------------------------------|
| ID                | Customer's unique identifier                              |
| Education         | Customer's education level                                |
| Age               | Customer's age                                            |
| Marital_Status    | Customer's marital status                                 |
| Kidhome           | Number of children in customer's household                |
| Teenhome          | Number of teenagers in customer's household               |
| Country           | Customer's country                                        |
| Recency           | Number of days since customer's last purchase             |
| Purchases         | Number of purchases made                                  |
| Spent_percent     | Spent % (of overall)                                      |
| Income            | Customer's yearly household income                        |
| CampaingsAccepted | Number of accepted campaings                              |
| Complain          | 1 if customer complained in the last 2 years, 0 otherwise |
| Spent             | How much the customer spent                               |

In [67]:
customers = pd.read_csv('campaings.csv')
customers.head()

,ID,Education,Age,DaysAsCustomer,Marital_Status,Kidhome,Teenhome,Country,Recency,Purchases,Spent_percent,Income,CampaingsAccepted,Complain,Spent
0,1826,Graduation,53,3392,Divorced,0,0,Spain,0,15,0.088457,84835.0,0,0,1190
1,1,Graduation,62,3393,Single,0,0,Canada,0,18,0.042891,57091.0,1,0,577
2,10476,Graduation,65,3426,Married,0,1,USA,0,11,0.018658,67267.0,0,0,251
3,1386,Graduation,56,3428,Together,1,1,Australia,0,4,0.000818,32474.0,0,0,11
4,5371,Graduation,34,3461,Single,1,0,Spain,0,8,0.006764,21474.0,1,0,91


In [68]:
customers.shape

(2213, 15)

In [69]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2213 entries, 0 to 2212
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 2213 non-null   int64  
 1   Education          2213 non-null   object 
 2   Age                2213 non-null   int64  
 3   DaysAsCustomer     2213 non-null   int64  
 4   Marital_Status     2213 non-null   object 
 5   Kidhome            2213 non-null   int64  
 6   Teenhome           2213 non-null   int64  
 7   Country            2213 non-null   object 
 8   Recency            2213 non-null   int64  
 9   Purchases          2213 non-null   int64  
 10  Spent_percent      2213 non-null   float64
 11   Income            2213 non-null   float64
 12  CampaingsAccepted  2213 non-null   int64  
 13  Complain           2213 non-null   int64  
 14  Spent              2213 non-null   int64  
dtypes: float64(2), int64(10), object(3)
memory usage: 259.5+ KB


In [70]:
customers.isna().sum() / len(customers) * 100

,0
ID,0.0
Education,0.0
Age,0.0
DaysAsCustomer,0.0
Marital_Status,0.0
Kidhome,0.0
Teenhome,0.0
Country,0.0
Recency,0.0
Purchases,0.0


In [71]:
customers.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,2213.0,5586.899232,3247.819194,0.000000,2815.000000,5455.000000,8420.000000,11191.000000
Age,2213.0,54.082693,11.700216,27.000000,46.000000,53.000000,64.000000,83.000000
DaysAsCustomer,2213.0,3732.731586,202.450745,3379.000000,3559.000000,3735.000000,3908.000000,4078.000000
Kidhome,2213.0,0.441934,0.536965,0.000000,0.000000,0.000000,1.000000,2.000000
Teenhome,2213.0,0.505648,0.544236,0.000000,0.000000,0.000000,1.000000,2.000000
Recency,2213.0,49.007682,28.941864,0.000000,24.000000,49.000000,74.000000,99.000000
Purchases,2213.0,14.889742,7.670341,0.000000,8.000000,15.000000,21.000000,44.000000
Spent_percent,2213.0,0.045122,0.044785,0.000372,0.005129,0.029511,0.077902,0.187693
Income,2213.0,52236.581563,25178.603047,1730.000000,35246.000000,51373.000000,68487.000000,666666.000000
CampaingsAccepted,2213.0,0.298238,0.679446,0.000000,0.000000,0.000000,0.000000,4.000000


In [72]:
customers.query('Age <= 90')

,ID,Education,Age,DaysAsCustomer,Marital_Status,Kidhome,Teenhome,Country,Recency,Purchases,Spent_percent,Income,CampaingsAccepted,Complain,Spent
0,1826,Graduation,53,3392,Divorced,0,0,Spain,0,15,0.088457,84835.0,0,0,1190
1,1,Graduation,62,3393,Single,0,0,Canada,0,18,0.042891,57091.0,1,0,577
2,10476,Graduation,65,3426,Married,0,1,USA,0,11,0.018658,67267.0,0,0,251
3,1386,Graduation,56,3428,Together,1,1,Australia,0,4,0.000818,32474.0,0,0,11
4,5371,Graduation,34,3461,Single,1,0,Spain,0,8,0.006764,21474.0,1,0,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,10142,PhD,47,3858,Divorced,0,1,USA,99,20,0.051216,66476.0,0,0,689
2209,5263,2n Cycle,46,3902,Married,1,0,Spain,99,5,0.004088,31056.0,0,0,55
2210,22,Graduation,47,3952,Divorced,1,0,Spain,99,14,0.022969,46310.0,0,0,309
2211,528,Graduation,45,3956,Married,0,0,India,99,20,0.102804,65819.0,0,0,1383


### **Features**

In [73]:
# Backup and features
customers = copy_dataframe(customers)
features = customers.copy()
features.drop(['ID', 'Country'], axis=1, inplace=True)

# Scale the selected columns and update 'features'
scaled_columns = [
    'Age', 'DaysAsCustomer', 'Kidhome', 'Teenhome',
    'Recency', 'Purchases', 'Spent_percent',
    ' Income ', 'CampaingsAccepted', 'Complain'
]
scaled_features = scale_features(features, scaled_columns)
features[scaled_columns] = scaled_features

# Encode the selected columns and update 'features'
encoded_columns = ['Education', 'Marital_Status']
encoded_features = label_encode_features(features, encoded_columns)
features[encoded_columns] = encoded_features[encoded_columns]

# 'features' now contains both scaled and encoded data
features.head(4)

,Education,Age,DaysAsCustomer,Marital_Status,Kidhome,Teenhome,Recency,Purchases,Spent_percent,Income,CampaingsAccepted,Complain,Spent
0,2,-0.092557,-1.683415,2,-0.823208,-0.929307,-1.693697,0.014378,0.967836,1.294980,-0.439042,-0.095498,1190
1,2,0.676833,-1.678474,4,-0.823208,-0.929307,-1.693697,0.405583,-0.049841,0.192843,1.033078,-0.095498,577
2,2,0.933297,-1.515435,3,-0.823208,0.908545,-1.693697,-0.507229,-0.591052,0.597087,-0.439042,-0.095498,251
3,2,0.163906,-1.505554,5,1.039532,0.908545,-1.693697,-1.420042,-0.989490,-0.785073,-0.439042,-0.095498,11


In [74]:
features.shape

(2213, 13)

In [75]:
customers.shape

(2213, 15)

### **Escolha do K**

In [76]:
# Selection on 'k'
elbow_silhouette_subplots(features[['Spent_percent', 'Age']], max_clusters=10)

Exmplo do Elbow:

Esse método usa o WCSS. Do inglês *within cluster sum of squares* ou soma dos quadrados dentro do cluster, o WCSS mede o quão próximo o centróide $x_c$ de um cluster está dos objectos alocados. O resultado pode ser interpretado com uma medida de proximidade dos objetos agrupados pois se o WCSS é baixo, os objetos estão próximos do centróide e, por consequência, próximos entre si. Matematicamente, é dado por:

\begin{equation}
  WCSS(x^k,x_c^k) = \sum_{i=1}^{n} (x_i^k-x_c^k)^{2}
\end{equation}

> No Python:

```python
# lista vazia
WCSS = []

# loop de iteração (teste com n clusters)
for k in range(1, 11):
  modelo_clustering = KMeans(n_clusters=k, random_state = 42)
  modelo_clustering = modelo_clustering.fit(dados)   # retorna os clusters
  WCSS.append(modelo_clustering.inertia_)             # armazena o WCSS na lista vazia

# visualização
with sns.axes_style('darkgrid'):
  grafico_ellbow = sns.lineplot(x=range(1, 11), y=WCSS, marker="8", color='k');
  grafico_ellbow.set(title='Método do Cotovelo', ylabel='WCSS', xlabel='Número de clusters');
```

### **Clusterização**

In [77]:
# Applying clustering
cluster_labels = apply_kmeans(features, k=4)
features['cluster'] = cluster_labels.astype(str)
customers['cluster'] = cluster_labels.astype(str)

### **Exploração dos clusters**

In [78]:
customers.columns

Index(['ID', 'Education', 'Age', 'DaysAsCustomer', 'Marital_Status', 'Kidhome',
       'Teenhome', 'Country', 'Recency', 'Purchases', 'Spent_percent',
       ' Income ', 'CampaingsAccepted', 'Complain', 'Spent', 'cluster'],
      dtype='object')

In [79]:
fig = px.scatter(
    customers,
    x = 'Spent',
    y = ' Income ',
    color = 'cluster',
    color_discrete_sequence = color_discrete_sequence
)

fig.update_traces(marker=custom_marker_style)

fig.update_layout(
    title='Gasots x Renda anual',
    width=700,
    height=400,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    title_x=0.45,
    legend_title='Cluster',
)
fig.show()

In [80]:
fig = px.box(
    customers,
    y = 'Spent',
    facet_col = 'Kidhome',
    color = 'cluster',
    color_discrete_sequence = color_discrete_sequence
)

fig.update_traces(marker=custom_marker_style)

fig.update_layout(
    title='Gastos x Crianças em casa',
    width=700,
    height=400,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    title_x=0.45,
    legend_title='Cluster',
)
fig.show()

In [81]:
fig = px.strip(
    customers.query('Marital_Status in ["Single", "Married", "Divorced"]'),
    y = 'Spent',
    facet_col = 'Marital_Status',
    color = 'cluster',
    color_discrete_sequence = color_discrete_sequence
)

fig.update_traces(marker=custom_marker_style)

fig.update_layout(
    title='Gastos x Crianças em casa',
    width=1000,
    height=400,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    title_x=0.45,
    legend_title='Cluster',
)
fig.show()

In [82]:
fig = px.strip(
    customers,
    y = 'Spent',
    facet_col = 'Teenhome',
    color = 'cluster',
    color_discrete_sequence = color_discrete_sequence
)

fig.update_traces(marker=custom_marker_style)

fig.update_layout(
    title='Gastos x Crianças em casa',
    width=1000,
    height=400,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    title_x=0.45,
    legend_title='Cluster',
)
fig.show()

In [83]:
fig = px.box(
    customers,
    y = 'Spent',
    color = 'cluster',
    color_discrete_sequence = color_discrete_sequence
)

fig.update_traces(marker=custom_marker_style)

fig.update_layout(
    title='Qual cluster gasta mais',
    width=700,
    height=400,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    title_x=0.45,
    legend_title='Cluster',
)
fig.show()

In [84]:
fig = px.strip(
    customers,
    y = 'Recency',
    facet_col='Kidhome',
    color = 'cluster',
    color_discrete_sequence = color_discrete_sequence
)

fig.update_traces(marker=custom_marker_style)

fig.update_layout(
    title='Recência por cluster e filhos pequenos em casa',
    width=700,
    height=400,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    title_x=0.45,
    legend_title='Cluster',
)
fig.show()

In [85]:
# Selecionar apenas as colunas numéricas e a coluna 'cluster'
clustered_customers = customers.copy()
encoded_columns = ['Education', 'Marital_Status']
clustered_customers = label_encode_features(clustered_customers, encoded_columns)
clustered_customers[encoded_columns] = clustered_customers[encoded_columns]
clustered_customers.drop(['Country', 'ID'], axis=1, inplace=True)
clustered_customers.groupby('cluster').median().style.background_gradient()

,Education,Age,DaysAsCustomer,Marital_Status,Kidhome,Teenhome,Recency,Purchases,Spent_percent,Income,CampaingsAccepted,Complain,Spent
cluster,,,,,,,,,,,,,
0,2.000000,55.000000,3749.000000,4.000000,0.000000,1.000000,46.000000,20.000000,0.041776,56601.500000,0.000000,0.000000,562.000000
1,2.000000,51.000000,3690.500000,4.000000,1.000000,0.000000,49.000000,7.000000,0.004906,34938.000000,0.000000,0.000000,66.000000
2,2.000000,53.000000,3812.500000,4.000000,0.000000,0.000000,53.000000,22.000000,0.126665,78945.500000,1.000000,0.000000,1704.000000
3,2.000000,57.000000,3793.000000,3.000000,0.000000,0.000000,53.000000,22.000000,0.082659,69759.000000,0.000000,0.000000,1112.000000


In [86]:
clustered_customers_summary =clustered_customers.groupby('cluster', as_index=False).median()
clustered_customers_summary['Ticket'] = clustered_customers_summary['Spent']/clustered_customers_summary['Purchases']
clustered_customers_summary[['cluster', 'Ticket']].style.bar(subset='Ticket', color='chartreuse')

,cluster,Ticket
0,0,28.100000
1,1,9.428571
2,2,77.454545
3,3,50.545455


In [87]:
clustered_customers.groupby('cluster').mean().style.background_gradient()

,Education,Age,DaysAsCustomer,Marital_Status,Kidhome,Teenhome,Recency,Purchases,Spent_percent,Income,CampaingsAccepted,Complain,Spent
cluster,,,,,,,,,,,,,
0,2.474178,55.859155,3746.866197,3.758216,0.288732,0.751174,47.032864,19.683099,0.042702,56322.462441,0.234742,0.011737,574.457746
1,2.289225,52.098299,3701.209830,3.712665,0.765595,0.486767,48.970699,8.133270,0.007476,36207.867675,0.085066,0.010397,100.571834
2,2.629630,54.407407,3783.270370,3.796296,0.051852,0.211111,51.566667,21.625926,0.131385,79114.966667,1.055556,0.000000,1767.496296
3,2.429194,56.816993,3762.542484,3.686275,0.067538,0.494553,49.420479,22.052288,0.083402,69580.002179,0.403050,0.008715,1121.984749


In [88]:
customers_by_clusters = customers.groupby('cluster', as_index=False)['ID'].count()
customers_by_clusters = customers_by_clusters.rename(columns={'ID': 'Customers'})
customers_by_clusters = customers_by_clusters.sort_values('Customers')
customers_by_clusters

,cluster,Customers
2,2,270
0,0,426
3,3,459
1,1,1058


**Dendograma (Clusterização Hierárquica)**

- Clusterização Hierárquica Algomerativa (bottom-up)
- Clusterização Hierárquica Divisiva (top-down)
- Métodos para o cálculo da distância.

<img src="https://github.com/OviedoVR/ML_especializacao/blob/main/assets/HC-linkages.png?raw=true" width="600px"/>

DOI: [10.1016/j.crfs.2023.100522](https://doi.org/10.1016/j.crfs.2023.100522)

Material complementar: [Hierarchical Clustering](https://www.learndatasci.com/glossary/hierarchical-clustering/)


### **Para os stakeholders**

Personas:

**Cluster 0:**
- Em torno de 54 anos
- Solteiro
- Sem filhos
- Grupo que mais gasta (média em torno de USD 1690)
- Aceita as campanhas enviadas
- Interessante para testes A/B
- Experimentação para estratégias de pricing

**Cluster 1:**
- Em torno de 52 anos
- Filhos em casa (pequenos)
- Recência e 49 dias e poucas compras (menor entre os grupos)
- Gastam menos (público menos rentável - 66 USD)
- Menor renda entre os grupos
- Oferecer produtos de baixo ticket pode ser interessante (vender em escala).


**Cluster 2:**
- 57 Anos
- Não tem filhos
- Segundo maior Ticket  (1102 USD)
- Não costuma aceitar campanhas
- Renda média-alta
- Oferecer promoções ou cashback pode ser interessante
- Não enviar muitos e-mails (estratégia de marketing para esse público)
- Buscar diminuir o valor da recência

**Cluster 3:**
- 56 anos
- Solteiro
- Filhos jovens em casa
- Menor Recência
- Compra relativamente com frequência similar aos clusters 0 e 2
- Gasta em torno de 561 USD
- Ticket de 28 USD
- Também não responde campanhas de marketing
- Aqui é interessante fidelizar bem este público, com programa de pontos, cashback, promoções com o valor beirando ao ticket, entender aonde esse público gasta
- Experimentação para estratégias de pricing


In [89]:
# renomeando a coluna cluster:
customers = customers.rename(columns={'cluster': 'Cluster'})
# elmininando a coluna Spent %:
customers.drop('Spent_percent', axis=1, inplace=True)

# categorizando os clusters (alfabeticamente):
customers['Cluster'] = customers['Cluster'].replace('1', 'A').replace('3', 'B').replace('0', 'C').replace('2', 'D')

# exportação dos dados:
customers.to_csv('segmented_customers.csv', index=False)

# visualização:
customers.head()

,ID,Education,Age,DaysAsCustomer,Marital_Status,Kidhome,Teenhome,Country,Recency,Purchases,Income,CampaingsAccepted,Complain,Spent,Cluster
0,1826,Graduation,53,3392,Divorced,0,0,Spain,0,15,84835.0,0,0,1190,B
1,1,Graduation,62,3393,Single,0,0,Canada,0,18,57091.0,1,0,577,C
2,10476,Graduation,65,3426,Married,0,1,USA,0,11,67267.0,0,0,251,A
3,1386,Graduation,56,3428,Together,1,1,Australia,0,4,32474.0,0,0,11,A
4,5371,Graduation,34,3461,Single,1,0,Spain,0,8,21474.0,1,0,91,A
